# AWS Re:Invent  Autogluon Workshop
### This workshop will demonstrate a machine learning problem solved by autogluon.
* Use the documentation of autogluon and the different tutorials [here](https://auto.gluon.ai/stable/index.html).

In [22]:
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

In [23]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [24]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [25]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20221019_114034/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221019_114034/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify po

In [26]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [27]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523
}


In [28]:
predictor.evaluate(test_data, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523
}


{'accuracy': 0.8374449790152523}

In [29]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.157960,0.062463,0.570961,0.157960,0.062463,0.570961,1,True,5
1,CatBoost,0.842461,0.85,0.037239,0.007632,0.915137,0.037239,0.007632,0.915137,1,True,7
2,RandomForestEntr,0.841130,0.83,0.159155,0.061881,0.477284,0.159155,0.061881,0.477284,1,True,6
3,LightGBM,0.839799,0.85,0.035789,0.005974,0.327178,0.035789,0.005974,0.327178,1,True,4
4,XGBoost,0.837445,0.87,0.040946,0.007789,0.218343,0.040946,0.007789,0.218343,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.044501,0.008325,0.596575,0.003556,0.000536,0.378232,2,True,14
6,LightGBMXT,0.836421,0.83,0.017048,0.005589,0.213307,0.017048,0.005589,0.213307,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.174446,0.060263,0.470668,0.174446,0.060263,0.470668,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.206226,0.060837,0.465740,0.206226,0.060837,0.465740,1,True,9
9,LightGBMLarge,0.828949,0.83,0.035526,0.011122,0.395346,0.035526,0.011122,0.395346,1,True,13


### Using hyperparameter settings
#### You can find [here](https://auto.gluon.ai/dev/api/autogluon.tabular.models.html) some models and which hyperparameters you can set.

In [30]:
import autogluon.core as ag

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


In [37]:
nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

rf_options = {
    'n_estimators': 400,
    'max_leaf_nodes' : ag.space.Int(lower=500, upper=15000, default=2000),
}

cat_options = {
    'learning_rate' : ag.space.Real(0.0,0.2,default=0.05),
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                   'RF': rf_options,
                   'CAT': cat_options,
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='acc').fit(
    train_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221019_115121/"
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221019_115121/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L1/T1 ...
	0.8713	 = Validation score   (accuracy)
	0.54s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	0.8715	 = Validation score   (accuracy)
	0.59s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	0.8694	 = Validation score   (accuracy)
	0.69s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	0.8193	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	0.8681	 = Validation score   (accuracy)
	0.64s	 = Training   runtime
	0.03s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L1 ... Tuning model for up to 11.99s of the 595.95s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: RandomForest_BAG_L1/T1 ...
	0.8639	 = Validation score   (accuracy)
	4.15s	 = Training   runtime
	1.51s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ... Tuning model for up to 11.99s of the 588.15s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 274.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L1/T1 ...
	0.8704	 = Validation score   (accuracy)
	9.54s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 11.99s of the 578.5s of remaining time.
2022-10-19 11:51:50,775	WARNING tune.py:682 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


KeyboardInterrupt: 

In [35]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8748080663322756
Evaluations on test data:
{
    "accuracy": 0.8748080663322756
}


In [36]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost/T5,0.875934,0.877000,0.021565,0.014523,14.451684,0.021565,0.014523,14.451684,1,True,15
1,LightGBM/T2,0.875934,0.876400,0.033536,0.045624,1.525269,0.033536,0.045624,1.525269,1,True,2
2,CatBoost/T3,0.875115,0.877600,0.020148,0.013745,11.598638,0.020148,0.013745,11.598638,1,True,13
3,CatBoost/T4,0.875115,0.877800,0.020232,0.013583,10.887210,0.020232,0.013583,10.887210,1,True,14
4,WeightedEnsemble_L2,0.874808,0.878800,1.067491,2.735195,81.948931,0.013306,0.007610,3.274111,2,True,21
5,CatBoost/T1,0.874706,0.876600,0.021638,0.013772,12.517243,0.021638,0.013772,12.517243,1,True,11
6,LightGBM/T1,0.874296,0.875600,0.039398,0.046588,1.945582,0.039398,0.046588,1.945582,1,True,1
7,CatBoost/T2,0.873989,0.877000,0.017486,0.012644,6.171221,0.017486,0.012644,6.171221,1,True,12
8,LightGBM/T3,0.873887,0.875600,0.050109,0.058517,1.698694,0.050109,0.058517,1.698694,1,True,3
9,LightGBM/T5,0.872249,0.874200,0.046430,0.036162,1.085283,0.046430,0.036162,1.085283,1,True,5


### Using ensembling techniques 

In [38]:
nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

rf_options = {
    'n_estimators': 400,
    'max_leaf_nodes' : ag.space.Int(lower=500, upper=15000, default=2000),
}

cat_options = {
    'learning_rate' : ag.space.Real(0.0,0.2,default=0.05),
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                   'RF': rf_options,
                   'CAT': cat_options,
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='acc').fit(
    train_data, time_limit=time_limit, #auto_stack=True,
    num_bag_folds=5, num_bag_sets=2, num_stack_levels=2,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221019_115218/"
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221019_115218/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM_BAG_L1/T1 ...
	0.8713	 = Validation score   (accuracy)
	0.55s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	0.8715	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	0.8694	 = Validation score   (accuracy)
	0.68s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	0.8193	 = Validation score   (accuracy)
	0.58s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	0.8681	 = Validation score   (accuracy)
	0.65s	 = Training   runtime
	0.03s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L1 ... Tuning model for up to 11.99s of the 596.0s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: RandomForest_BAG_L1/T1 ...
	0.8637	 = Validation score   (accuracy)
	3.82s	 = Training   runtime
	1.39s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ... Tuning model for up to 11.99s of the 588.75s of remaining time.


...

Fitted model: LightGBM_BAG_L3/T1 ...
	0.8816	 = Validation score   (accuracy)
	0.64s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L3/T2 ...
	0.8801	 = Validation score   (accuracy)
	0.67s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L3/T3 ...
	0.8811	 = Validation score   (accuracy)
	0.82s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM_BAG_L3/T4 ...
	0.8514	 = Validation score   (accuracy)
	0.69s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM_BAG_L3/T5 ...
	0.8807	 = Validation score   (accuracy)
	0.75s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L3 ... Tuning model for up to 9.72s of the 211.67s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: RandomForest_BAG_L3/T1 ...
	0.8728	 = Validation score   (accuracy)
	7.24s	 = Training   runtime
	1.21s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L3 ... Tuning model for up to 9.72s of the 201.53s of remaining time.


  0%|          | 0/5 [00:00<?, ?it/s]

	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L3/T1 ...
	0.8819	 = Validation score   (accuracy)
	7.65s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L3 ... Tuning model for up to 9.72s of the 193.75s of remaining time.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
2022-10-19 11:59:13,840	INFO stopper.py:363 -- Reached timeout of 7.772111286163332 seconds. Stopping all trials.
Fitted model: NeuralNetTorch_BAG_L3/T1 ...
	0.8796	 = Validation score   (accuracy)
	4.0s	 = Training   runtime
	0.13s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L3/T2 ...
	0.8802	 = Validation score   (accuracy)
	3.76s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: LightGBM_BAG_L3/T1 ... Training model for up to 184.23s of the 184.22s of remaining time.
	Fitting 4 child models (S1F2 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.8752	 = Valida

In [39]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8745009724639164
Evaluations on test data:
{
    "accuracy": 0.8745009724639164
}


In [40]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L2/T1,0.876344,0.873647,4.607929,6.392114,207.766691,0.130898,0.145210,3.098280,2,True,12
1,LightGBM_BAG_L2/T5,0.876241,0.873519,4.621554,6.435337,208.331747,0.144523,0.188432,3.663336,2,True,16
2,CatBoost_BAG_L2/T1,0.876036,0.875259,4.536136,6.349077,228.112616,0.059106,0.102173,23.444205,2,True,18
3,NeuralNetTorch_BAG_L2/T2,0.876036,0.873800,5.286805,7.319544,233.640086,0.809774,1.072639,28.971675,2,True,20
4,LightGBM_BAG_L2/T2,0.875934,0.874594,4.596604,6.382348,208.082765,0.119574,0.135443,3.414354,2,True,13
5,NeuralNetTorch_BAG_L3/T1,0.875729,0.874389,8.779307,11.338098,341.504479,1.406430,1.038578,29.374693,3,True,29
6,LightGBM_BAG_L3/T1,0.875627,0.875234,7.513283,10.468890,315.592884,0.140405,0.169370,3.463098,3,True,22
7,CatBoost_BAG_L1/T1,0.875525,0.874415,0.099878,0.165818,90.194755,0.099878,0.165818,90.194755,1,True,7
8,WeightedEnsemble_L2,0.875525,0.874415,0.101681,0.224807,101.035081,0.001802,0.058988,10.840326,2,True,11
9,WeightedEnsemble_L3,0.875525,0.875797,6.286457,8.940844,286.408552,0.003397,0.056937,9.914819,3,True,21


#### Extra automation

In [42]:
nn_options = {}
gbm_options = {}
rf_options = {}
cat_options = {}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                   'RF': rf_options,
                   'CAT': cat_options,
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 10*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric='acc').fit(
    train_data, time_limit=time_limit, auto_stack=True, presets='best_quality',
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20221019_120848/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels/ag-20221019_120848/"
AutoGluon Version:  0.5.2
Python Version:     3.8.12
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-

  0%|          | 0/5 [00:00<?, ?it/s]

[1000]	valid_set's binary_error: 0.125281


Fitted model: LightGBM_BAG_L1/T1 ...
	0.8747	 = Validation score   (accuracy)
	0.97s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T2 ...
	0.8745	 = Validation score   (accuracy)
	1.1s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T3 ...
	0.8755	 = Validation score   (accuracy)
	0.97s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T4 ...
	0.8747	 = Validation score   (accuracy)
	4.68s	 = Training   runtime
	0.26s	 = Validation runtime
Fitted model: LightGBM_BAG_L1/T5 ...
	0.8759	 = Validation score   (accuracy)
	0.75s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: RandomForest_BAG_L1 ... Tuning model for up to 16.87s of the 589.61s of remaining time.
	No hyperparameter search space specified for RandomForest. Skipping HPO. Will train one model based on the provided hyperparameters.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/en

  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 381.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L1/T1 ...
	0.8743	 = Validation score   (accuracy)
	13.43s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch_BAG_L1 ... Tuning model for up to 16.87s of the 569.74s of remaining time.
NaN or Inf found in input tensor.
2022-10-19 12:09:33,280	INFO stopper.py:363 -- Reached timeout of 13.494065542817118 seconds. Stopping all trials.
Fitted model: NeuralNetTorch_BAG_L1/T1 ...
	0.8485	 = Validation score   (accuracy)
	9.56s	 = Training   runtime
	0.08s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T2 ...
	0.8506	 = Validation score   (accuracy)
	10.0s	 = Training   runtime
	0.08s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T3 ...
	0.8356	 = Validation score   (accuracy)
	8.2s	 = Training   runtime
	0.17s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L1/T4 ...
	0.8346	 = Validation score   (accura

In [43]:
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=False)

Evaluation: accuracy on test data: 0.8749104309550619
Evaluations on test data:
{
    "accuracy": 0.8749104309550619
}


In [44]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1/T1,0.876446,0.873647,0.145938,0.252939,128.013721,0.145938,0.252939,128.013721,1,True,6
1,LightGBM_BAG_L1/T1,0.876036,0.874543,0.416960,0.577250,8.500129,0.416960,0.577250,8.500129,1,True,1
2,LightGBM_BAG_L1/T4,0.875525,0.873800,2.969287,4.153718,25.714271,2.969287,4.153718,25.714271,1,True,4
3,LightGBM_BAG_L1/T3,0.875422,0.873979,0.457224,0.593544,9.099113,0.457224,0.593544,9.099113,1,True,3
4,LightGBM_BAG_L1/T2,0.875320,0.874850,0.410834,0.473358,8.757940,0.410834,0.473358,8.757940,1,True,2
5,WeightedEnsemble_L2,0.874910,0.874978,5.659217,6.664397,225.251606,0.003643,0.058712,9.797962,2,True,11
6,LightGBM_BAG_L1/T5,0.874194,0.874568,0.390232,0.475660,8.644814,0.390232,0.475660,8.644814,1,True,5
7,NeuralNetTorch_BAG_L1/T1,0.862217,0.858061,1.468261,0.925699,163.836490,1.468261,0.925699,163.836490,1,True,7
8,NeuralNetTorch_BAG_L1/T2,0.859863,0.855066,1.344663,0.980691,144.756043,1.344663,0.980691,144.756043,1,True,8
9,NeuralNetTorch_BAG_L1/T3,0.854028,0.847874,4.660900,2.944922,74.556832,4.660900,2.944922,74.556832,1,True,9
